In [1]:
#####################################
#####################################
######## import package #############
#####################################
#####################################
import numpy as np
from scipy.stats import skew, skewtest
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Train shape is:", train.shape)
print("Test shape is:", test.shape)


Train shape is: (1460, 81)
Test shape is: (1459, 80)


In [3]:
#delete outliers
#1stFlrSF
train = train.drop(train[(train['1stFlrSF']>4000)].index)
#BedroomAbvGr
train = train.drop(train[(train['BedroomAbvGr']==8)].index)
#BsmtFinSF1
train = train.drop(train[(train['BsmtFinSF1']>5000)].index)
#GrLivArea
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)
#LotArea
train = train.drop(train[(train['LotArea']>100000)].index)
#LotFrontage
train = train.drop(train[(train['LotFrontage']>300)].index)
#OpenPorchSF
train = train.drop(train[(train['OpenPorchSF']>500)&(train['SalePrice']<100000)].index)
#TotalBsmtSF
train = train.drop(train[(train['TotalBsmtSF']>6000)].index)

In [4]:
train.shape

(1451, 81)

In [5]:
total = pd.concat((train, test)).reset_index(drop=True)
print("Total shape is:", total.shape)

Total shape is: (2910, 81)


In [17]:
char_X = total.select_dtypes(include=["object"])
print(char_X.shape)
np.sum(char_X.isnull())

(2910, 23)


BldgType         0
CentralAir       0
Condition1       0
Condition2       0
Electrical       0
Exterior1st      0
Exterior2nd      0
Foundation       0
GarageType       0
Heating          0
HouseStyle       0
LandContour      0
LotConfig        0
MSSubClass       0
MSZoning         0
MasVnrType       0
MiscFeature      0
MoSold           0
Neighborhood     0
RoofMatl         0
RoofStyle        0
SaleCondition    0
SaleType         0
dtype: int64

In [7]:
#######################################
######### Missing Values ##################
#######################################
### Categorical Data
total["PoolQC"] = total["PoolQC"].fillna("None")
total["MiscFeature"] = total["MiscFeature"].fillna("None")
total["Alley"] = total["Alley"].fillna("None")
total["Fence"] = total["Fence"].fillna("None")
total["FireplaceQu"] = total["FireplaceQu"].fillna("None")
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
total["LotFrontage"] = total.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    total[col] = total[col].fillna('None')
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    total[col] = total[col].fillna('None')
total["MasVnrType"] = total["MasVnrType"].fillna("None")
total["MSZoning"] = total.groupby("Neighborhood")["MSZoning"].transform(
    lambda x: x.fillna(x.mode()[0]))##
total["Utilities"] = total.groupby("Neighborhood")["Utilities"].transform(
    lambda x: x.fillna(x.mode()[0]))##
total["Functional"] = total["Functional"].fillna("Typ")
total['Electrical'] = total['Electrical'].fillna(total['Electrical'].mode()[0])
total['KitchenQual'] = total['KitchenQual'].fillna("TA")
total['Exterior1st'] = total['Exterior1st'].fillna(total['Exterior1st'].mode()[0])
total['Exterior2nd'] = total['Exterior2nd'].fillna(total['Exterior2nd'].mode()[0])
total['SaleType'] = total['SaleType'].fillna("Oth")

In [18]:
cont_X = total.select_dtypes(include=["float64"])
print(cont_X.shape)
np.sum(np.isnan(cont_X))

(2910, 25)


1stFlrSF            0
2ndFlrSF            0
3SsnPorch           0
BsmtFinSF1          0
BsmtFinSF2          0
BsmtFullBath        0
BsmtHalfBath        0
BsmtUnfSF           0
EnclosedPorch       0
GarageArea          0
GarageCars          0
GarageYrBlt         0
GrLivArea           0
KitchenAbvGr        0
LotArea             0
LotFrontage         0
LowQualFinSF        0
MasVnrArea          0
MiscVal             0
OpenPorchSF         0
PoolArea            0
SalePrice        1459
ScreenPorch         0
TotalBsmtSF         0
WoodDeckSF          0
dtype: int64

In [9]:
#######################################
######### Missing Values ##################
#######################################
### Continuous Data
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageArea', 'GarageCars'):
    total[col] = total[col].fillna(0)
total["LotFrontage"] = total.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
total["MasVnrArea"] = total["MasVnrArea"].fillna(0)
total["GarageYrBlt"] = total["GarageYrBlt"].fillna(total["YearBuilt"])

##################################################
########### Integer to Character #################
##################################################
total['MSSubClass'] = total['MSSubClass'].apply(str)
total['MoSold'] = total['MoSold'].astype(str)

In [10]:
#***************************************************
#****** Transformation *****************************
#***************************************************

# sqrt root transformation
total['BsmtUnfSF'] = total['BsmtUnfSF'].apply(np.sqrt)
# log1p transformation
total[['BsmtFinSF1', 'BsmtFinSF2', "MasVnrArea", "SalePrice"]] = total[['BsmtFinSF1', 'BsmtFinSF2', "MasVnrArea", "SalePrice"]].apply(np.log1p)
intFeature = total.select_dtypes(include=["int64"]).apply(lambda x: skew(x.astype(float).dropna()))
skewIntFeat = intFeature[intFeature > .75].index.tolist()
total[skewIntFeat] = total[skewIntFeat].apply(np.log1p)

In [11]:
#*****************************************************
#******* Character to Ordinal ************************
#*****************************************************

total = total.replace({"Alley" : {"None" : 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"None" : 0, "No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4},##
                       "BsmtFinType1" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "Fence" : {"None" : 0, "MnWw" : 1, "GdWo": 2, "MnPrv": 3, "GdPrv": 4},
                       "FireplaceQu" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},##
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageFinish" : {"None" : 0, "Unf" : 1, "RFn" : 2, "Fin" : 3},
                       "GarageQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"None" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}})

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
catCol = total.select_dtypes(["object"]).columns.tolist()
print(len(catCol))
total.select_dtypes(["object"]).dtypes
LabelMap = dict()
for i in catCol:
    print('Start encoding ' + i)
    lb = preprocessing.LabelBinarizer()
    lb.fit(total[i].astype(str))
    LabelMap[i] = dict(zip(lb.classes_, lb.transform(lb.classes_)))
    preprocessing.LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
    binarydata = lb.transform(total[i])
    for j in range(len(lb.classes_)-1):
        coln = i + '_' + lb.classes_[j]
        total[coln]=binarydata[:,[j]]

23
Start encoding BldgType
Start encoding CentralAir
Start encoding Condition1
Start encoding Condition2
Start encoding Electrical
Start encoding Exterior1st
Start encoding Exterior2nd
Start encoding Foundation
Start encoding GarageType
Start encoding Heating
Start encoding HouseStyle
Start encoding LandContour
Start encoding LotConfig
Start encoding MSSubClass
Start encoding MSZoning
Start encoding MasVnrType
Start encoding MiscFeature
Start encoding MoSold
Start encoding Neighborhood
Start encoding RoofMatl
Start encoding RoofStyle
Start encoding SaleCondition
Start encoding SaleType


In [15]:
new_train = total[:train.shape[0]]
new_test = total[train.shape[0]:]

In [32]:
#new_train.to_csv("new_train2.csv", index=False)
#new_test.to_csv("new_test2.csv", index=False)

In [40]:
#total.to_csv("total.csv", index=False)

In [16]:
# drop ID columns
total.drop("Id", axis=1, inplace=True)

In [13]:
total.shape

(2910, 249)

In [16]:
new_test.shape

(1459, 249)